In [105]:
import struct, os
from Crypto.Cipher import AES
DNS_URL_POSTFIX = "dnsr.uk.to"
CODING_TABLE    = "IQJ6WORKEGL5B4YS3NZMPDHVCUA7FX2T"
ct = {}
for i,c in enumerate(CODING_TABLE.upper()):
    ct[c] = i
#SECRET_KEY = os.urandom(32)  #That's better.
qn = "LHL4JWRWWOYQOP44UGTSZLPJTR.ENOHH3Y24PFTAXEN5IN4GNB7FQ.V5PV2VSNXCEKYGIY4JKGOVBPJJ.CPFRZBVBZGRA3LT42OXIPUB67Q.dnsr.uk.to."

### Add this to dnsSneaker.h

In [67]:
print("#define SECRET_KEY_256							{", end="")
for c in SECRET_KEY:
    print( "0x{0:02x},".format(c), end="")
print("}")

#define SECRET_KEY_256							{0xbe,0xa5,0x93,0x8e,0x43,0x4f,0x65,0xef,0x42,0xe7,0x11,0x45,0xc1,0xa0,0xfb,0x48,0xbf,0x8c,0x1a,0xf0,0x9b,0x36,0x88,0x4e,0x6d,0x6c,0x20,0x25,0xb4,0xf5,0xae,0x7c,}


### This is where the magic happens

In [68]:
def decodeBlock( messageBlock ):
    """ returns a bytes object """
    global ct
    res = 0
    for i,c in enumerate(messageBlock):
        res |= ct[c]<<(i*5)
    return res.to_bytes( 16, 'little' )

In [69]:
def dnsDecode( qnString ):
    global DNS_URL_POSTFIX, SECRET_KEY
    cipher = AES.new( SECRET_KEY, AES.MODE_ECB )
    if not qnString.endswith( '.' + DNS_URL_POSTFIX + '.'):
        raise RuntimeError("Bad hostname" + qnString.split(".")[-1])
    messageBlocks = qn.replace( '.' + DNS_URL_POSTFIX + '.', "" )
    resultBytes = bytearray()
    for messageBlock in messageBlocks.split("."):
        messageBlock.upper()
        resultBytes += cipher.decrypt( decodeBlock(messageBlock) )
    plLength = resultBytes[-3]
    plCrc = resultBytes[-2]<<8 | resultBytes[-1]
    return resultBytes

In [106]:
res = dnsDecode( qn )
print(res.decode())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa6 in position 62: invalid start byte

In [107]:
def hexdump( res ):
    for i,b in enumerate(res):
        if( len(res)>16 and (i%16)==0 ):
            print( "\n  {:04x}: ".format(i), end="" )
        print( "{:02x} ".format(b), end="" )

In [108]:
hexdump(res)


  0000: 57 75 68 75 75 21 2c 20 43 6f 6d 6d 75 6e 69 63 
  0010: 61 74 69 6f 6e 20 73 65 65 6d 73 20 66 69 6e 65 
  0020: 21 21 20 3a 44 20 3a 44 20 53 6f 20 63 6f 6f 6c 
  0030: 21 20 0a 09 2e 2e 2e 0a 00 00 00 00 00 39 a6 13 

In [111]:
class Crc16:
    def __init__( self ):
        self._resetRunnningCRC()
    def _resetRunnningCRC( self ):
        self.c=0xFFFF
    def _runningCRC( self, inputByte ):
        self.c ^= inputByte
        self.c &= 0xFFFF
        for b in range(8):             # For each bit in the byte
            if self.c & 1:
                self.c = (self.c >> 1) ^ 0xA001
            else: 
                self.c = (self.c >> 1)
            self.c &= 0xFFFF
    def getCrc( self, dataBytes ):
        self._resetRunnningCRC()
        for b in dataBytes:
            self._runningCRC( b )
        return self.c

In [136]:
class DnsDecoder:
    def __init__( self, SECRET_KEY, SUBDOMAIN='.dnsr.uk.to.', CODING_TABLE="IQJ6WORKEGL5B4YS3NZMPDHVCUA7FX2T" ):
        self.CODING_TABLE = CODING_TABLE
        self.SUBDOMAIN = SUBDOMAIN
        self.decodingTable = {}
        for i,c in enumerate(CODING_TABLE.upper()):
            self.decodingTable[c] = i
        self.cipher = AES.new( SECRET_KEY, AES.MODE_ECB )
        self.crc = Crc16()
    
    def _decodeBlock( self, messageBlock ):
        """ returns a bytes object """
        res = 0
        for i,c in enumerate(messageBlock):
            res |= self.decodingTable[c]<<(i*5)
        return res.to_bytes( 16, 'little' )
    
    def dnsDecode( self, qnString ):
        """ decode URL string and return payload as bytes """
        if not qnString.endswith( self.SUBDOMAIN ):
            raise RuntimeError("Bad hostname: " + qnString)
        messageBlocks = qnString.replace( self.SUBDOMAIN, "" )
        resultBytes = bytearray()
        for messageBlock in messageBlocks.split("."):
            messageBlock.upper()
            resultBytes += self.cipher.decrypt( self._decodeBlock(messageBlock) )
        plCrc = resultBytes[-2]<<8 | resultBytes[-1]
        plLength = resultBytes[-3]
        resultBytes = resultBytes[:plLength]
        if self.crc.getCrc( resultBytes ) != plCrc:
            raise RuntimeError("CRC error")
        return resultBytes

In [137]:
dc = DnsDecoder( SECRET_KEY )

In [148]:
hexdump( dc.dnsDecode("6V3QMI5QLFKZGHAZ4ZU44LX6BW.5ZOUGBMD3XSSFMSUAMZROZ6ZNJ.WCPQHK35VK2OROL4TCYOKYCU7J.dnsr.uk.to.") )


  0000: 00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 
  0010: 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 

In [149]:
6*8

48